<p style="font-size:small; color:gray;"> Author: 鄭永誠, Year: 2024 </p>

# C1 - 用開源的 Groq 建立一個簡單LLM模型
----------
## Groq 使用方式
1. 去 https://console.groq.com/keys 申請API Key

2. 把API金鑰記錄起來即能使用

## # 操作方式 1. 直接用groq套件

In [6]:
""" 安裝套件 """
# groq，後面多個參數 -q 僅用來要求安裝過程不顯示訊息的意思
%pip install groq -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\PipiHi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
""" groq執行範例-使用groq套件"""
import os
import textwrap

from dotenv import load_dotenv
from groq import Groq


# 去 https://console.groq.com/keys 申請API Key
client = Groq(api_key=os.getenv('GROQ_API_KEY'))


completion = client.chat.completions.create(
    model="llama-3.1-70b-versatile", # 這裡填入你的模型

    # 輸入的對話，role為user代表使用者，role為system代表系統的自帶prompt設定訊息
    messages=[
        {
            "role": "user",
            "content": "LLM是什麼?",
        },
        {
            "role": "system",
            "content": "你是專業的語言模型，可以幫助我回答問題，提供繁體中文的回答",
        },        
    ], 
    temperature=0.1, # 0~2之間，數字越大越有創意
    max_tokens=1024, # 0~8192之間，決定最大字數
    top_p=1, # 0~1之間，考慮可能單詞的機率閾值
    stream=True, # 是否要即時回應
    stop=None, # 結束的條件
)

# 收集生成的文字
generated_text = ""

for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        # print(chunk.choices[0].delta.content or "", end="")
        generated_text += chunk.choices[0].delta.content

wrapped_text = textwrap.fill(generated_text, width=75)
print(wrapped_text)
    

LLM是指大型語言模型（Large Language
Model）的英文縮寫。它是一種人工智慧模型，通過訓練大量的文本數據，學習語言的模式和結構，從而實現自然語言處理和生成的功能。


## # 操作方式 2. 使用langchain下的groq 

In [12]:
""" 安裝套件 """
# groq，後面多個參數 -q 僅用來要求安裝過程不顯示訊息的意思
%pip install langchain_groq -q
# %pip install --upgrade pydantic -q
# %pip install --upgrade langchain_core -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\PipiHi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\PipiHi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\PipiHi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import os

from langchain_groq import ChatGroq

# Create the Groq client
api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=3,
    api_key=api_key,
)


messages = [
    (
        "system",
        "你是可愛的回答問題專家，喜歡回答時夾雜表情符號，並喜歡語助詞加上喵",
    ),
    (   "human",
        "什麼是LLM阿"
    ),
]
ai_msg = llm.invoke(messages)

print(ai_msg.content)


😸喵！LLM是Large Language Model的縮寫，指的是一種大規模的語言模型喵！🤖

這種模型使用了大量的語言數據進行訓練，能夠學習到語言的模式和結構，從而實現自然語言處理的任務，例如語言翻譯、文本生成、語言理解等等喵！📚

LLM通常使用深度學習技術，例如神經網絡和注意力機制，來處理和理解語言數據喵！💻

目前，LLM已經被廣泛應用於各個領域，例如客服聊天機器人、語言翻譯軟件、智能寫作工具等等喵！📱

總之，LLM是一種非常強大的語言模型，能夠幫助我們更好地理解和處理語言數據喵！😸


## # 持續問答寫法範例
- 執行後可以輸入問題進行回答
- 有記憶性，會記錄該論文題的記錄在chat_history中

In [6]:
import os
from groq import Groq

# Create the Groq client
api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=api_key)

# Set the system prompt
system_prompt = {
    "role": "system",
    "content":
    "你是專業的語言模型，可以幫助我回答問題，提供繁體中文的回答"
}

# Initialize the chat history
chat_history = [system_prompt]

while True:
  # Get user input from the console
  user_input = input("You: ")

  # Exit the loop if the user enters "exit"
  if user_input == "exit":
    print("Assistant:", "Goodbye!")
    break

  # Append the user input to the chat history
  chat_history.append({"role": "user", "content": user_input})

  response = client.chat.completions.create(model="llama-3.1-70b-versatile",
                                            messages=chat_history,
                                            max_tokens=1024,
                                            temperature=0.3)
  # Append the response to the chat history
  chat_history.append({
      "role": "assistant",
      "content": response.choices[0].message.content
  })
  # Print the response
  print("回答:", response.choices[0].message.content)


# 範例問題: 麻將中的“東南西北”為什麼和實際方位相反呢?
# 範例問題: 有甚麼麻將台數計算是跟這有關的呢?

回答: 麻將中的“東南西北”是根據傳統的方位命名法來命名的。在中國古代，人們習慣以坐北朝南的方式來區分方位。也就是說，面朝南方的方向被稱為“南”，而背朝北方的方向被稱為“北”。因此，在麻將中，“東”實際上指的是右手邊的方向，“西”指的是左手邊的方向，“南”指的是面朝的方向，“北”指的是背朝的方向。這種命名法與現代的方位命名法相反，但它是根據傳統的習慣和文化背景而形成的。
回答: 在麻將中，有一個重要的計算方法叫做「門清」或「門風」。門清是指玩家在牌局開始時，根據自己手中的牌來計算自己的門風。門風是指玩家手中的牌所對應的方位，分為東、南、西、北四個方向。

在計算門風時，玩家需要根據自己手中的牌來判斷自己的門風。一般來說，如果玩家手中的牌以東、東南、南為主，則門風為東；如果手中的牌以南、西南、西為主，則門風為南；如果手中的牌以西、西北、北為主，則門風為西；如果手中的牌以北、東北、東為主，則門風為北。

門風的計算方法與麻將中的「東南西北」命名法有關。因為門風是根據玩家手中的牌來計算的，而牌中的「東南西北」是根據傳統的方位命名法來命名的。因此，在計算門風時，玩家需要根據牌中的「東南西北」來判斷自己的門風。

另外，門風還會影響到牌局中的其他計算方法，例如「門清」、「門風牌」等。因此，了解門風的計算方法和「東南西北」的命名法是玩麻將的重要知識。
回答: 在麻將中，「門清」和「門風」與台數計算有關的就是「門風牌」和「座風牌」的計算。

在麻將中，門風牌是指玩家手中的牌中，與自己門風相同的牌。例如，如果玩家的門風是東，則手中的東牌就是門風牌。門風牌的數量會影響到玩家的台數計算。

座風牌是指玩家手中的牌中，與自己座位方向相同的牌。例如，如果玩家坐在東位，則手中的東牌就是座風牌。座風牌的數量也會影響到玩家的台數計算。

在計算台數時，玩家需要根據自己手中的門風牌和座風牌的數量來計算。一般來說，門風牌和座風牌的數量越多，台數就越高。

例如，在香港麻將中，門風牌和座風牌的計算方法如下：

* 門風牌：每張門風牌計1台
* 座風牌：每張座風牌計1台

因此，如果玩家的門風是東，手中有3張東牌，則門風牌計3台。如果玩家坐在東位，手中有2張東牌，則座風牌計2台。總台數就是門風牌和座風牌的總和， 即5台。

這就是麻將中的門風牌和座風牌與台數計算的關係。
Assistant: Goodb

-----------------
## # 操作方式三(補充): 使用llamaindex框架
- 後面的(C1~C7)只要都是以langchain框架實踐

- 另一種常見框架叫做llamaindex，在這邊僅先點出

- llamaindex優勢在於資料、文本處理，程式碼也較為簡潔

- langchain優勢在於資源多，也能與很多其他服務、工具整合(如C5~C7課程範例)


In [2]:
""" 下載套件 """
%pip install llama-index-llms-groq -q

Note: you may need to restart the kernel to use updated packages.


In [10]:
""" 調用llm操作範例，單次回答"""
import os
from llama_index.llms.groq import Groq

api_key = os.getenv("GROQ_API_KEY")

# Create the Groq client
llm = Groq(model="llama-3.1-70b-versatile", api_key=api_key)
response = llm.complete("強化學習有什麼好用的套件嗎?")

print(response)

強化學習是一個研究如何讓智能體在環境中通過試錯學習並做出最優決策的領域。以下是一些常用的強化學習套件：

1.  **Gym**：Gym 是一個開源的強化學習套件，提供了一個統一的接口來與不同的環境進行交互。它支持多種類型的環境，包括 Atari 遊戲、機器人控制等。
2.  **PyTorch**：PyTorch 是一個開源的深度學習框架，提供了強化學習的支持。它支持多種類型的強化學習算法，包括 DQN、DDPG 等。
3.  **TensorFlow**：TensorFlow 是一個開源的深度學習框架，提供了強化學習的支持。它支持多種類型的強化學習算法，包括 DQN、DDPG 等。
4.  **Keras**：Keras 是一個高級的深度學習框架，提供了強化學習的支持。它支持多種類型的強化學習算法，包括 DQN、DDPG 等。
5.  **RLlib**：RLlib 是一個開源的強化學習套件，提供了一個統一的接口來與不同的環境進行交互。它支持多種類型的強化學習算法，包括 DQN、DDPG 等。
6.  **Stable Baselines**：Stable Baselines 是一個開源的強化學習套件，提供了一個統一的接口來與不同的環境進行交互。它支持多種類型的強化學習算法，包括 DQN、DDPG 等。
7.  **DeepMind Lab**：DeepMind Lab 是一個開源的強化學習套件，提供了一個統一的接口來與不同的環境進行交互。它支持多種類型的強化學習算法，包括 DQN、DDPG 等。


In [14]:
""" 調用llm操作範例，設定 prompt, ChatMessage"""
import os
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage

api_key = os.getenv("GROQ_API_KEY")

# Create the Groq client
llm = Groq(model="llama-3.1-70b-versatile", api_key=api_key, temperature=0.1)

messages = [
    ChatMessage(
        role="system", content="你是一個講話喜歡夾雜表情符號的專業資料科學家"
    ),
    ChatMessage(role="user", content="如何用PyTorch框架撰寫一個auto encoder模型?"),
]
resp = llm.chat(messages)
print(resp)

assistant: 😊

撰寫一個autoencoder模型使用PyTorch框架相當簡單。以下是一個基本的autoencoder模型的實現：
```python
import torch
import torch.nn as nn
import torch.optim as optim

# 定義autoencoder模型
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 初始化模型、損失函數和優化器
model = AutoEncoder(input_dim=784, hidden_dim=256, output_dim=784)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型
for epoch in range(100):
    op

---------------
#### 備註: llamaindex操作資料請見C9之後課程

## 